The code might not be working due to the paths being of my local drive and in our college IDs we cant create shared drives
 -SK

In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F

Ensuring reproduciblity

In [ ]:
import random,numpy as np
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

Getting the Data

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_set = datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_set = datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# Print the sizes of the datasets
print("Training set size:", len(train_set))
print("Testing set size:", len(test_set))


100%|██████████| 9912422/9912422 [00:00<00:00, 207818288.92it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 30306653.45it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 60890581.86it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 12959543.38it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Training set size: 60000
Testing set size: 10000


In [ ]:
torch.manual_seed(42)
initial_train_set,rest = torch.utils.data.random_split(train_set,[10000,len(train_set)-10000])
val_set,remainder = torch.utils.data.random_split(rest,[10000,len(rest)-10000])
len(initial_train_set),len(rest),len(remainder),len(val_set)

(10000, 50000, 40000, 10000)

The LeNet-5 model

In [ ]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = LeNet5()
model

LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

Training loop

In [ ]:
def train_model(model, train_dataset, epochs, learning_rate):
    train_loader = DataLoader(train_dataset,batch_size = 64)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader, 0):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss / len(train_loader):.3f}")

    print("Finished training")

# train_model(model,initial_train_set,10,0.01)

Test loop

In [ ]:
def test_model(model, test_set):
    test_loader = DataLoader(test_set,batch_size=64)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy
test_model(model,test_set)

96.57

Training and saving the the model with best accuracy

In [ ]:
def train_test_save(train_dataset, test_dataset,n, epochs, lr):
    test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=64)
    train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=64)
    best_accuracy = 0.0
    model = LeNet5()
    train_model(model, train_dataset, epochs, lr)
    best_accuracy = test_model(model, test_set)
    print(f"Initial best accuracy:{best_accuracy:.2f}")

    torch.save(model.state_dict(), f"/content/drive/MyDrive/MyML/Models/model_{best_accuracy:.2f}.pt")

    for _ in range(n):
        model=LeNet5()
        train_model(model, train_dataset, epochs, lr)
        new_accuracy = test_model(model, test_set)
        print(f"New accuracy:{new_accuracy:.2f}\nBest accuracy:{best_accuracy:.2f}")
        if new_accuracy > best_accuracy :
            best_accuracy = new_accuracy
            if new_accuracy > 95.00 :
              torch.save(model.state_dict(), f"/content/drive/MyDrive/MyML/Models/model_{best_accuracy:.2f}.pt")

train_test_save(initial_train_set, test_set, n=3, epochs=10, lr=0.01)

Epoch [1/10] Loss: 2.300
Epoch [2/10] Loss: 2.290
Epoch [3/10] Loss: 2.271
Epoch [4/10] Loss: 2.207
Epoch [5/10] Loss: 1.766
Epoch [6/10] Loss: 0.792
Epoch [7/10] Loss: 0.495
Epoch [8/10] Loss: 0.387
Epoch [9/10] Loss: 0.322
Epoch [10/10] Loss: 0.277
Finished training
Initial best accuracy:92.20
Epoch [1/10] Loss: 2.299
Epoch [2/10] Loss: 2.278
Epoch [3/10] Loss: 2.228
Epoch [4/10] Loss: 1.902
Epoch [5/10] Loss: 0.844
Epoch [6/10] Loss: 0.465
Epoch [7/10] Loss: 0.356
Epoch [8/10] Loss: 0.299
Epoch [9/10] Loss: 0.261
Epoch [10/10] Loss: 0.233
Finished training
New accuracy:92.02
Best accuracy:92.20
Epoch [1/10] Loss: 2.301
Epoch [2/10] Loss: 2.290
Epoch [3/10] Loss: 2.271
Epoch [4/10] Loss: 2.214
Epoch [5/10] Loss: 1.804
Epoch [6/10] Loss: 0.768
Epoch [7/10] Loss: 0.462
Epoch [8/10] Loss: 0.360
Epoch [9/10] Loss: 0.307
Epoch [10/10] Loss: 0.273
Finished training
New accuracy:91.22
Best accuracy:92.20
Epoch [1/10] Loss: 2.300
Epoch [2/10] Loss: 2.289
Epoch [3/10] Loss: 2.270
Epoch [4/10]

Retrieving and loading the model with best accuracy

In [ ]:
import os

folder_path = "/content/drive/MyDrive/MyML/Models/"

def get_accuracy_from_filename(filename):
    return float(filename.split("_")[1][:-3])
model_files = os.listdir(folder_path)
model_files = [file for file in model_files if file.startswith("model_") and file.endswith(".pt")]

if not model_files:
    print("No model files found in the folder.")
else:
    # Find the model file with the highest accuracy
    best_model_filename = max(model_files, key=get_accuracy_from_filename)
    best_model_path = os.path.join(folder_path, best_model_filename)
    best_model = LeNet5()
    best_model.load_state_dict(torch.load(best_model_path))

    print(f"Loaded the model with the highest accuracy: {best_model_path}")

Loaded the model with the highest accuracy: /content/drive/MyDrive/MyML/Models/model_96.57.pt


Least Confidence and retraining

In [ ]:
from torch.utils.data import DataLoader, Subset

def least_confidence_images(model, test_dataset, k=5000):
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    confidences = []
    labels = []
    with torch.no_grad():
        for images, targets in test_loader:
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            max_probs, _ = torch.max(probs, dim=1)
            confidences.extend(max_probs.tolist())
            labels.extend(targets.tolist())
    confidences = torch.tensor(confidences)
    _, indices = torch.topk(confidences, k, largest=False)
    return Subset(test_dataset, indices), [labels[i] for i in indices]
least_conf_images, least_conf_labels = least_confidence_images(model, remainder, k=5000)
train_dataset = torch.utils.data.ConcatDataset([initial_train_set, least_conf_images])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
len(train_dataset)

15000

In [ ]:
def train_until_degradation(model, initial_train_set, remaining_set, val_set, batch_size, degradation_threshold=0.05):
    # Initialize the training dataset with the initial subset
    train_dataset = initial_train_set
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set,batch_size=64)
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    # Initial evaluation on the validation set to calculate baseline accuracy
    best_accuracy = test_model(model, val_set)
    current_accuracy = best_accuracy
    # Initialize the degradation flag
    is_degraded = False
    epoch = 0
    max_epochs = 100  # Set the maximum number of epochs to 10
    while not is_degraded and epoch < max_epochs:
        epoch += 1
        model.train()
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
        least_conf_images, _ = least_confidence_images(model, remaining_set, k=5000)
        # Combine the least confident images with the current training dataset
        train_dataset = torch.utils.data.ConcatDataset([train_dataset, least_conf_images])
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        current_accuracy = test_model(model, val_set)
        print(f"Epoch [{epoch}], Current accuracy: {current_accuracy:.2f}, Best accuracy: {best_accuracy:.2f}")
        # Check for improvement in accuracy
        if current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
        # Check for degradation
        if (best_accuracy - current_accuracy) >= degradation_threshold:
            is_degraded = True
            print("Degradation threshold reached. Stopping training.")
    print("Final accuracy on validation set:", current_accuracy)
    torch.save(model, f"/content/drive/MyDrive/MyML/Models/final_model_{current_accuracy}.pt")
    return model
# new_model = train_until_degradation(model, initial_train_set, remainder, val_set, batch_size=64, degradation_threshold=0.05)

Epoch [1], Current accuracy: 96.5900, Best accuracy: 96.3500
Epoch [2], Current accuracy: 98.0300, Best accuracy: 96.5900
Epoch [3], Current accuracy: 98.1100, Best accuracy: 98.0300
Epoch [4], Current accuracy: 98.7100, Best accuracy: 98.1100
Epoch [5], Current accuracy: 98.8100, Best accuracy: 98.7100
Epoch [6], Current accuracy: 98.8600, Best accuracy: 98.8100
Epoch [7], Current accuracy: 99.0600, Best accuracy: 98.8600
Epoch [8], Current accuracy: 98.7500, Best accuracy: 99.0600
Degradation threshold reached. Stopping training.
Final accuracy on validation set: 98.75
